# 0.2.0 Simple topic identification

In [2]:
%load_ext autoreload
%autoreload 2

In [6]:
import pandas as pd
from matplotlib import pyplot as plt
import re

from nltk.tokenize import sent_tokenize, word_tokenize, regexp_tokenize, TweetTokenizer
from collections import Counter

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import warnings
warnings.filterwarnings("ignore")

In [7]:
import sys
sys.path.append("../") 

from utils.info import article
import utils.paths as path
from utils.paths2 import direcciones, direcciones_cursos

## 0.1.2 Bag-of-words picker

In [5]:
Counter(word_tokenize("""The cat is in the box. The cat box."""))

Counter({'The': 2, 'cat': 2, 'is': 1, 'in': 1, 'the': 1, 'box': 2, '.': 2})

## 0.1.3 Building a Counter with bag-of-words

In [9]:
# Tokenize the article: tokens
tokens = word_tokenize(article)
tokens

["''",
 "'Debugging",
 "''",
 "'",
 'is',
 'the',
 'process',
 'of',
 'finding',
 'and',
 'resolving',
 'of',
 'defects',
 'that',
 'prevent',
 'correct',
 'operation',
 'of',
 'computer',
 'software',
 'or',
 'a',
 'system',
 '.',
 'Numerous',
 'books',
 'have',
 'been',
 'written',
 'about',
 'debugging',
 '(',
 'see',
 'below',
 ':',
 '#',
 'Further',
 'reading|Further',
 'reading',
 ')',
 ',',
 'as',
 'it',
 'involves',
 'numerous',
 'aspects',
 ',',
 'including',
 'interactive',
 'debugging',
 ',',
 'control',
 'flow',
 ',',
 'integration',
 'testing',
 ',',
 'Logfile|log',
 'files',
 ',',
 'monitoring',
 '(',
 'Application',
 'monitoring|application',
 ',',
 'System',
 'Monitoring|system',
 ')',
 ',',
 'memory',
 'dumps',
 ',',
 'Profiling',
 '(',
 'computer',
 'programming',
 ')',
 '|profiling',
 ',',
 'Statistical',
 'Process',
 'Control',
 ',',
 'and',
 'special',
 'design',
 'tactics',
 'to',
 'improve',
 'detection',
 'while',
 'simplifying',
 'changes',
 '.',
 'Origin',
 'A

In [13]:
# Convert the tokens into lowercase: lower_tokens
lower_tokens = [t.lower() for t in tokens]
lower_tokens

["''",
 "'debugging",
 "''",
 "'",
 'is',
 'the',
 'process',
 'of',
 'finding',
 'and',
 'resolving',
 'of',
 'defects',
 'that',
 'prevent',
 'correct',
 'operation',
 'of',
 'computer',
 'software',
 'or',
 'a',
 'system',
 '.',
 'numerous',
 'books',
 'have',
 'been',
 'written',
 'about',
 'debugging',
 '(',
 'see',
 'below',
 ':',
 '#',
 'further',
 'reading|further',
 'reading',
 ')',
 ',',
 'as',
 'it',
 'involves',
 'numerous',
 'aspects',
 ',',
 'including',
 'interactive',
 'debugging',
 ',',
 'control',
 'flow',
 ',',
 'integration',
 'testing',
 ',',
 'logfile|log',
 'files',
 ',',
 'monitoring',
 '(',
 'application',
 'monitoring|application',
 ',',
 'system',
 'monitoring|system',
 ')',
 ',',
 'memory',
 'dumps',
 ',',
 'profiling',
 '(',
 'computer',
 'programming',
 ')',
 '|profiling',
 ',',
 'statistical',
 'process',
 'control',
 ',',
 'and',
 'special',
 'design',
 'tactics',
 'to',
 'improve',
 'detection',
 'while',
 'simplifying',
 'changes',
 '.',
 'origin',
 'a

In [14]:
# Create a Counter with the lowercase tokens: bow_simple
bow_simple = Counter(lower_tokens)
bow_simple

Counter({"''": 69,
         "'debugging": 1,
         "'": 2,
         'is': 25,
         'the': 150,
         'process': 12,
         'of': 81,
         'finding': 1,
         'and': 41,
         'resolving': 1,
         'defects': 3,
         'that': 14,
         'prevent': 1,
         'correct': 1,
         'operation': 2,
         'computer': 12,
         'software': 16,
         'or': 25,
         'a': 60,
         'system': 19,
         '.': 89,
         'numerous': 2,
         'books': 1,
         'have': 7,
         'been': 3,
         'written': 1,
         'about': 3,
         'debugging': 39,
         '(': 40,
         'see': 5,
         'below': 1,
         ':': 31,
         '#': 1,
         'further': 1,
         'reading|further': 1,
         'reading': 1,
         ')': 40,
         ',': 151,
         'as': 21,
         'it': 18,
         'involves': 1,
         'aspects': 1,
         'including': 1,
         'interactive': 3,
         'control': 4,
         'flow': 2,
  

In [15]:
# Print the 10 most common tokens
print(bow_simple.most_common(10))

[(',', 151), ('the', 150), ('.', 89), ('of', 81), ("''", 69), ('to', 63), ('a', 60), ('``', 47), ('in', 44), ('and', 41)]


## 0.1.4 Text preprocessing steps

Which of the following are useful text preprocessing steps?

R:/ Lemmatization, lowercasing, removing unwanted tokens.

## 0.1.5 Text preprocessing practice

In [33]:
print('ok_')

ok_
